In [68]:
from scipy.stats import norm
import numpy as np
import yfinance as yf
import pandas as pd

from datetime import datetime


def get_historical_data(stk, start_date, end_date):
    df = yf.download(stk, start_date, end_date, interval="1wk").dropna()
    return df

def trade_parameters(df):

    df['change_dollar']=df.Close-df.Open
    df['change_pct']=(df.Close - df.Open)/df.Open
    
    df['abs_change']=abs(df.Close-df.Open)

    df['abs_change_roll2']=df.abs_change.rolling(2).sum()
    df['abs_change_roll3']=df.abs_change.rolling(3).sum()
    df['abs_change_roll4']=df.abs_change.rolling(4).sum()
     
    df['abs_change_rank_shift']=df.abs_change.rank(pct=True).shift(1)
    df['abs_change_rank_r2_shift']=df.abs_change_roll2.rank(pct=True).shift(1)
    df['abs_change_rank_r3_shift']=df.abs_change_roll3.rank(pct=True).shift(1)
    df['abs_change_rank_r4_shift']=df.abs_change_roll4.rank(pct=True).shift(1)

    
    df['abs_change_roll3']=abs(df.Close-df.Open).rolling(3).sum()
    df['abs_change_pct']=(abs((df.Close - df.Open)/df.Open))
    
    
    df['volume_change']=(df.Volume-df.Volume.shift(1))
    df['volume_change_pct']=(df.Volume-df.Volume.shift(1))/df.Volume.shift(1)

    df=df.iloc[4:,::]
    #df['combo1']=(df.abs_change.pow(2)+df.abs_change_roll2.pow(2)).pow(1/2)
    #df['combo2']=(df.abs_change+df.abs_change_roll2)/2
    #print(df)
    
    
    #df['abs_change_prev_roll_2s']=df.abs_change.rolling(2).sum().shift(1)
    #df['abs_change_prev']=df.abs_change.shift(1)
    
   # df['abs_change_rank']=df.abs_change.rank(pct=True)
    
    #df['change_rank_prev']=df.change_rank.shift(1)
    
    lim=0.8
    df=df[df.abs_change_rank_shift<lim]
    df=df[df.abs_change_rank_r2_shift<lim]
    df=df[df.abs_change_rank_r3_shift<lim]
    df=df[df.abs_change_rank_r4_shift<lim]
    
    #df=df[df.change_rank1<0.20]
    return df



start_date = "2000-01-01"
end_date = "2022-08-22"
arr = []
stock = "SPY"
df = get_historical_data(stock, start_date, end_date)
df = trade_parameters(df)
#df.to_csv(stock + ".csv")
# vol_rank=df.volume_rank.iloc[-1]
#lr_rank=df.change_rank.iloc[-1]


avg_abs_dollar_change=np.average(df.abs_change)
periods=len(df.index)
cor_1=np.corrcoef(df.change_dollar.iloc[2:],df.change_dollar.shift(1).iloc[2:])[0,1]
cor_2=np.corrcoef(df.change_pct.iloc[2:],df.change_pct.shift(1).iloc[2:])[0,1]

cor_3=np.corrcoef(df.abs_change.iloc[2:],df.abs_change.shift(1).iloc[2:])[0,1]
cor_4=np.corrcoef(df.abs_change_pct.iloc[2:],df.abs_change_pct.shift(1).iloc[2:])[0,1]

cor_5=np.corrcoef(df.abs_change.iloc[2:],df.volume_change_pct.shift(1).iloc[2:])[0,1]

#cor_5=np.corrcoef(df.Volume.iloc[2:],df.Volume.shift(1).iloc[2:])[0,1]
#cor_5=np.corrcoef(df.Volume,df.abs_change)[0,1]
#cor_5=np.corrcoef(df.abs_change.iloc[2:],df.Volume.shift(1).iloc[2:])[0,1]

cor_6=np.corrcoef(df.abs_change.iloc[2:],df.abs_change.rolling(2).sum().shift(1).iloc[2:])[0,1]
cor_7=np.corrcoef(df.abs_change.iloc[3:],df.abs_change.rolling(3).sum().shift(1).iloc[3:])[0,1]
cor_8=np.corrcoef(df.abs_change.iloc[4:],df.abs_change.rolling(4).sum().shift(1).iloc[4:])[0,1]


#cor_9=np.corrcoef(df.abs_change.iloc[2:],df.combo1.shift(1).iloc[2:])[0,1]

#cor_2=np.corrcoef(df.abs_change_pct.iloc[2:], df.Volume.shift(1).iloc[2:])[0,1]
#cor_3=np.corrcoef(df.abs_change_pct.iloc[3:],df.abs_change_pct.rolling(3).sum().shift(1).iloc[3:])[0,1]
#cor_4=np.corrcoef(df.abs_change_pct.iloc[2:],
#                  df.abs_change_pct.shift(1).iloc[2:] 
#                  * df.Volume.shift(1).iloc[2:])[0,1]
#cor_5=np.corrcoef(df.abs_change_pct.iloc[2:],
 #                 (df.abs_change_pct.shift(1).iloc[2:] * 
#                  df.abs_change_pct.rolling(2).sum().shift(1).iloc[2:])/2)[0,1]
arr.append(
    [
        stock,
        avg_abs_dollar_change,
        periods,
        cor_1,
        cor_2,
        cor_3,
        cor_4,
        cor_5,
        cor_6,
        cor_7,
        cor_8,

    ]
)

finaldf = pd.DataFrame(
data=arr,
columns=[
    "stock",
    'avg_abs_dollar_change',
    'periods',
    'cor_1',
    'cor_2',
    'cor_3',
    'cor_4',
    'cor_5',
    'cor_6',
    'cor_7',
    'cor_8',

],
)

#finaldf.to_csv("big.csv")
print(finaldf)
#print(np.average(finaldf.cor_1.dropna()))
#print(np.average(finaldf.cor_2.dropna()))
#print(np.average(finaldf.cor_3.dropna()))
#print(np.average(finaldf.cor_4.dropna()))
#print(np.average(finaldf.cor_5.dropna()))
#print(np.average(finaldf.cor_5.dropna()))

[*********************100%***********************]  1 of 1 completed
  stock  avg_abs_dollar_change  periods     cor_1     cor_2     cor_3  \
0   SPY               2.073182      817  0.074234  0.085247  0.150843   

      cor_4     cor_5     cor_6     cor_7     cor_8  
0  0.178383  0.036974  0.211657  0.239686  0.238502  


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins